# Quantifying uncertainty in weather data
CityLearn comes with several dimenstions of weather data. Namely: actual hourly measurements as well as 6-,12-, and 24-hour point estimate forecasts for Outdoor Drybulb Temperature, Relative Humidity, Diffuse and Direct Solar Radiation.

The goal of this subproject is to identify the distributions of these point estimates, giving us the opportunity to know when to rely how much on these forecasts.

## Loading the Data

In [1]:
# canonical imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# let's read in the data (this would probably be easier in SQL, but I don't want to set up a database)
weather = pd.read_csv("data/citylearn_challenge_2022_phase_1/weather.csv")
# the buildings contain per-row time data
building1 = pd.read_csv("data/citylearn_challenge_2022_phase_1/Building_1.csv")


In [4]:
# let's look at the weather data
weather

,Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],6h Prediction Outdoor Drybulb Temperature [C],12h Prediction Outdoor Drybulb Temperature [C],24h Prediction Outdoor Drybulb Temperature [C],6h Prediction Relative Humidity [%],12h Prediction Relative Humidity [%],24h Prediction Relative Humidity [%],6h Prediction Diffuse Solar Radiation [W/m2],12h Prediction Diffuse Solar Radiation [W/m2],24h Prediction Diffuse Solar Radiation [W/m2],6h Prediction Direct Solar Radiation [W/m2],12h Prediction Direct Solar Radiation [W/m2],24h Prediction Direct Solar Radiation [W/m2]
0,20.0,84.0,0.0,0.0,18.3,22.8,20.0,81.0,68.0,81.0,25.0,964.0,0.0,100.0,815.0,0.0
1,20.1,79.0,0.0,0.0,19.4,22.8,19.4,79.0,71.0,87.0,201.0,966.0,0.0,444.0,747.0,0.0
2,19.7,78.0,0.0,0.0,21.1,22.2,19.4,73.0,73.0,87.0,420.0,683.0,0.0,592.0,291.0,0.0
3,19.3,78.0,0.0,0.0,22.2,22.8,19.4,71.0,71.0,90.0,554.0,522.0,0.0,491.0,153.0,0.0
4,18.9,78.0,0.0,0.0,21.7,22.2,18.9,73.0,71.0,90.0,778.0,444.0,0.0,734.0,174.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,20.6,84.0,26.0,130.0,20.1,19.4,20.6,79.0,79.0,73.0,0.0,201.0,27.0,0.0,444.0,143.0
8756,21.1,81.0,0.0,0.0,19.7,21.1,20.0,78.0,73.0,76.0,0.0,420.0,0.0,0.0,592.0,0.0
8757,21.7,79.0,0.0,0.0,19.3,22.2,20.6,78.0,71.0,70.0,0.0,554.0,0.0,0.0,491.0,0.0
8758,21.3,76.0,0.0,0.0,18.9,21.7,20.6,78.0,73.0,73.0,0.0,778.0,0.0,0.0,734.0,0.0


In [38]:
# Let's declare some helper variables to deal with the column names
pred_6 = "6h Prediction "
pred_12 = "12h Prediction "
pred_24 = "24h Prediction "

true_6 = "6h Lookahead "
true_12 = "12h Lookahead "
true_24 = "24h Lookahead "

temperature = "Outdoor Drybulb Temperature [C]"
humidity = "Relative Humidity [%]"
diffuse = "Diffuse Solar Radiation [W/m2]"
direct = "Direct Solar Radiation [W/m2]"

# quick sanity check
assert pred_6+temperature == "6h Prediction Outdoor Drybulb Temperature [C]"

In [20]:
# Next, let's integrate the date information from the building data.
# The next line shows us that we don't have to deal with Daylight Savings Time:
building1[building1["Daylight Savings Status"] != 0] # returns 0 rows

,Month,Hour,Day Type,Daylight Savings Status,Indoor Temperature [C],Average Unmet Cooling Setpoint Difference [C],Indoor Relative Humidity [%],Equipment Electric Power [kWh],DHW Heating [kWh],Cooling Load [kWh],Heating Load [kWh],Solar Generation [W/kW]


In [29]:
# This means we can omit this column, and simply add Month, Hour and Day Type to our weather dataframe:
weathertime = building1[["Month", "Hour", "Day Type"]].join(weather)
weathertime.head()

,Month,Hour,Day Type,Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],6h Prediction Outdoor Drybulb Temperature [C],12h Prediction Outdoor Drybulb Temperature [C],24h Prediction Outdoor Drybulb Temperature [C],6h Prediction Relative Humidity [%],12h Prediction Relative Humidity [%],24h Prediction Relative Humidity [%],6h Prediction Diffuse Solar Radiation [W/m2],12h Prediction Diffuse Solar Radiation [W/m2],24h Prediction Diffuse Solar Radiation [W/m2],6h Prediction Direct Solar Radiation [W/m2],12h Prediction Direct Solar Radiation [W/m2],24h Prediction Direct Solar Radiation [W/m2]
0,8.0,0.0,1.0,20.0,84.0,0.0,0.0,18.3,22.8,20.0,81.0,68.0,81.0,25.0,964.0,0.0,100.0,815.0,0.0
1,8.0,1.0,1.0,20.1,79.0,0.0,0.0,19.4,22.8,19.4,79.0,71.0,87.0,201.0,966.0,0.0,444.0,747.0,0.0
2,8.0,2.0,1.0,19.7,78.0,0.0,0.0,21.1,22.2,19.4,73.0,73.0,87.0,420.0,683.0,0.0,592.0,291.0,0.0
3,8.0,3.0,1.0,19.3,78.0,0.0,0.0,22.2,22.8,19.4,71.0,71.0,90.0,554.0,522.0,0.0,491.0,153.0,0.0
4,8.0,4.0,1.0,18.9,78.0,0.0,0.0,21.7,22.2,18.9,73.0,71.0,90.0,778.0,444.0,0.0,734.0,174.0,0.0


In [48]:
# Also, let's add the actual future values so we have something to compare the predictions to.
df = weathertime[:24] #remove last day of predictions, which has no ground truth for 24h predicitons
#actuals = pd.DataFrame()
df[true_6+temperature] = weathertime[6:-18][temperature]
df[true_12+temperature] = weathertime[12:-12][temperature]

df


/var/folders/xk/q5r2r6d96n34m06fwvg751s40000gn/T/ipykernel_26892/3755607727.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[true_6+temperature] = weathertime[6:-18][temperature]
/var/folders/xk/q5r2r6d96n34m06fwvg751s40000gn/T/ipykernel_26892/3755607727.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[true_12+temperature] = weathertime[12:-12][temperature]


,Month,Hour,Day Type,Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],6h Prediction Outdoor Drybulb Temperature [C],12h Prediction Outdoor Drybulb Temperature [C],24h Prediction Outdoor Drybulb Temperature [C],...,12h Prediction Relative Humidity [%],24h Prediction Relative Humidity [%],6h Prediction Diffuse Solar Radiation [W/m2],12h Prediction Diffuse Solar Radiation [W/m2],24h Prediction Diffuse Solar Radiation [W/m2],6h Prediction Direct Solar Radiation [W/m2],12h Prediction Direct Solar Radiation [W/m2],24h Prediction Direct Solar Radiation [W/m2],6h Lookahead Outdoor Drybulb Temperature [C],12h Lookahead Outdoor Drybulb Temperature [C]
0,8.0,0.0,1.0,20.0,84.0,0.0,0.0,18.3,22.8,20.0,...,68.0,81.0,25.0,964.0,0.0,100.0,815.0,0.0,NaN,NaN
1,8.0,1.0,1.0,20.1,79.0,0.0,0.0,19.4,22.8,19.4,...,71.0,87.0,201.0,966.0,0.0,444.0,747.0,0.0,NaN,NaN
2,8.0,2.0,1.0,19.7,78.0,0.0,0.0,21.1,22.2,19.4,...,73.0,87.0,420.0,683.0,0.0,592.0,291.0,0.0,NaN,NaN
3,8.0,3.0,1.0,19.3,78.0,0.0,0.0,22.2,22.8,19.4,...,71.0,90.0,554.0,522.0,0.0,491.0,153.0,0.0,NaN,NaN
4,8.0,4.0,1.0,18.9,78.0,0.0,0.0,21.7,22.2,18.9,...,71.0,90.0,778.0,444.0,0.0,734.0,174.0,0.0,NaN,NaN
5,8.0,5.0,1.0,18.3,81.0,0.0,0.0,22.2,21.1,18.3,...,76.0,93.0,903.0,392.0,0.0,811.0,396.0,0.0,NaN,NaN
6,8.0,6.0,1.0,18.3,81.0,25.0,100.0,22.8,20.6,18.9,...,78.0,90.0,964.0,231.0,15.0,815.0,320.0,14.0,18.3,NaN
7,8.0,7.0,1.0,19.4,79.0,201.0,444.0,22.8,20.6,20.0,...,73.0,87.0,966.0,27.0,116.0,747.0,143.0,42.0,19.4,NaN
8,8.0,8.0,1.0,21.1,73.0,420.0,592.0,22.2,20.0,20.0,...,76.0,87.0,683.0,0.0,252.0,291.0,0.0,74.0,21.1,NaN
9,8.0,9.0,1.0,22.2,71.0,554.0,491.0,22.8,20.6,20.0,...,70.0,84.0,522.0,0.0,292.0,153.0,0.0,35.0,22.2,NaN
